In [10]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [11]:
fn = './tables/TableS3.csv'
df = pd.read_csv(fn)
region = 'global'

In [17]:
plt.close('all')
def plot_geodesics(df, region, opacity_s=0.8):
    print("Plotting geodesics")
    #df = df.query('Dest in @destinations')
    fig = plt.figure(figsize=vis.MAP_SIZE)
    plateCr = ccrs.PlateCarree()
    plateCr._threshold = plateCr._threshold/10.
    ax = plt.axes(projection=plateCr)
    
    #mx = df.Prediction.max()
    if region == 'global':
        cutoff = 0.005
    else:
        cutoff = 0.1
    opacity = np.maximum(df.risk_ij, cutoff)
    lines = plt.plot(df[['origin_lon', 'dest_lon']].T,
                     df[['origin_lat', 'dest_lat']].T, 
                     color='r',
                     transform=ccrs.Geodetic())
    print('scalin')
    [line.set_alpha(vis.opacity_scaler(alpha, opacity_s)) for alpha, line in zip(opacity, lines)]
    
    vis._annotate(ax, text='a', color='k', region=region)
    vis._add_features(ax, region=region)

    plt.tight_layout()
plot_geodesics(df, region, opacity_s=1.5)

Plotting geodesics
scalin


masked_array(data=[0.52656971, 0.66219197, 0.98766947, ..., 0.15271373,
                   0.01485872, 0.03986964],
             mask=False,
       fill_value=1e+20)

In [43]:
plt.close('all')
x = np.linspace(0,1)
f = lambda x,s=1: np.maximum(np.arctan(1/(1-x)**s - 1/x**s) / np.pi + 0.5, 0.05)
y = f(x)
plt.plot(x,y)
p = 0.75
plt.scatter(p, f(p),color='r')
plt.show()
f(p)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


0.8857997487800919

In [31]:
y

array([0.        , 0.00663338, 0.01356146, 0.02081828, 0.0284436 ,
       0.03648417, 0.04499516, 0.0540421 , 0.06370317, 0.07407217,
       0.0852622 , 0.09741033, 0.11068332, 0.12528468, 0.14146303,
       0.15952135, 0.17982617, 0.20281377, 0.22898749, 0.25889544,
       0.29307025, 0.33190868, 0.3754773 , 0.42327455, 0.47406225,
       0.52593775, 0.57672545, 0.6245227 , 0.66809132, 0.70692975,
       0.74110456, 0.77101251, 0.79718623, 0.82017383, 0.84047865,
       0.85853697, 0.87471532, 0.88931668, 0.90258967, 0.9147378 ,
       0.92592783, 0.93629683, 0.9459579 , 0.95500484, 0.96351583,
       0.9715564 , 0.97918172, 0.98643854, 0.99336662, 1.        ])

In [ ]:
x**2 + (y-1)**2 = 1
y-1 = -(1-x**2) + 1